# Create TTL Reflecting the WordNet Analysis to Match New Nouns/Verbs

In [1]:
# Map all the nouns and verbs associated with only 1 class (noun-sing.txt and verb-sing.txt) into TTL
def get_turtle(file_text: str, is_verb: bool) -> list:
    word_maps = file_text.split("\n")
    new_turtle = []
    for wmap in word_maps:
        word = wmap.split(" ['")[0]
        class_name = wmap.split("['")[1]
        class_name = class_name.split("']")[0]
        if is_verb:
            new_turtle.append(f'<{class_name}> :verb_synonym "{word}" .')
        else:
            new_turtle.append(f'<{class_name}> :noun_synonym "{word}" .')
    return new_turtle
    
with open('new_synonyms.ttl', 'w') as out_file:
    with open("verb-sing.txt", "r") as f:   # Mapping of verb to DNA class name
        f_text = f.read()
    out_file.write('\n'.join(get_turtle(f_text, True)))
    with open("noun-sing.txt", "r") as f:   # Mapping of noun to DNA class name
        f_text = f.read()
    out_file.write('\n'.join(get_turtle(f_text, False)))

In [4]:
# Map all the multiple choice and/or multiple type nouns/verbs (noun-multi.txt and verb-multi.txt)
#   into a dictionary format and then pickle the dictionary
import pickle

def get_mappings(file_text: str, is_verb: bool, word_dict: dict):
    word_maps = file_text.split("\n")
    for wmap in word_maps:
        word = wmap.split(" ['")[0]
        if word in word_dict.keys():
            print('dup term: ', word)
            break
        class_names = wmap.split("['")[1].split("']")[0]
        indiv_names = class_names.split("', '")
        for indiv_name in indiv_names:
            if not indiv_name.startswith('urn:') and indiv_name != '+do':
                print('not urn:', word, indiv_name)
        word_dict[word] = indiv_names

verbs_multiple = dict()
with open("verb-multi.txt", "r") as f:   # Mapping of verb to DNA class names
    f_text = f.read()
get_mappings(f_text, True, verbs_multiple)   
nouns_multiple = dict()
with open("noun-multi.txt", "r") as f:   # Mapping of verb to DNA class names
    f_text = f.read()
get_mappings(f_text, True, nouns_multiple) 


In [5]:
print(verbs_multiple)

{'order': ['urn:ontoinsights:dna:Alignment', 'urn:ontoinsights:dna:CommandAndDemand', 'urn:ontoinsights:dna:PurchaseAndSale'], 'exercise': ['urn:ontoinsights:dna:BodyMovement+urn:ontoinsights:dna:HealthAndDiseaseRelated', 'urn:ontoinsights:dna:UtilizationAndConsumption+urn:ontoinsights:dna:NegPerson'], 'go along with': ['urn:ontoinsights:dna:MovementTravelAndTransportation+urn:ontoinsights:dna:Person', 'urn:ontoinsights:dna:Agreement'], 'set up': ['urn:ontoinsights:dna:ProductionManufactureAndCreation', 'urn:ontoinsights:dna:Betrayal'], 'expire': ['urn:ontoinsights:dna:End', 'urn:ontoinsights:dna:Death'], 'close': ['urn:ontoinsights:dna:End', 'urn:ontoinsights:dna:EndOfBusiness'], 'run off': ['urn:ontoinsights:dna:MovementTravelAndTransportation+urn:ontoinsights:dna:BodyMovement'], 'endorse': ['urn:ontoinsights:dna:FriendlyOrSupportiveAct+urn:ontoinsights:dna:CommunicationAndSpeechAct'], 'record': ['urn:ontoinsights:dna:ProductionManufactureAndCreation+urn:ontoinsights:dna:InformationS

In [6]:
with open('nouns-multiple.pickle', 'wb') as out_file:
    pickle.dump(nouns_multiple, out_file)
with open('verbs-multiple.pickle', 'wb') as out_file:
    pickle.dump(verbs_multiple, out_file)

# Check for Duplicates in Multi-Type Nouns and Verbs

In [5]:
import stardog

sd_conn_details = {'endpoint': 'http://localhost:5820',
                   'username': 'admin',
                   'password': 'admin'}

query_noun_syn = 'prefix : <urn:ontoinsights:dna:> select ?class where {?class :noun_synonym "term" }'
query_verb_syn = 'prefix urn: <urn:ontoinsights:dna:> select ?class where {?class :verb_synonym "term" }'

dna_conn = stardog.Connection("dna", **sd_conn_details)

In [6]:
with open('verb-multi.txt', 'r') as in_file:
    f_text = in_file.read()
multi_records = f_text.split('\n')
for rec in multi_records:
    texts = rec.split(" ['")
    syn_results = dna_conn.select(query_verb_syn.replace("term", texts[0]), 
                                  content_type='application/sparql-results+json')
    for syn_result in syn_results['results']['bindings']:
        class_name = syn_result['class']['value']
        print('text: ', texts[0], 'class: ', class_name, 'mapping: ', texts[1])
     

In [7]:
with open('noun-multi.txt', 'r') as in_file:
    f_text = in_file.read()
multi_records = f_text.split('\n')
for rec in multi_records:
    texts = rec.split(" ['")
    syn_results = dna_conn.select(query_noun_syn.replace("term", texts[0]), 
                                  content_type='application/sparql-results+json')
    for syn_result in syn_results['results']['bindings']:
        class_name = syn_result['class']['value']
        print('text: ', texts[0], 'class: ', class_name, 'mapping: ', texts[1])